# Kafka Streaming avec PySpark

Ce notebook permet de consommer des messages Kafka en temps réel avec Spark Structured Streaming.


## Imports et dépendances

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

## Configuration des constantes

Définition des paramètres de connexion Kafka et des versions des packages.

In [2]:
SPARK_VERSION = "4.0.1" 
BROKER_ADDRESS = "broker:29092"
TOPIC = "test" 
STARTING_OFFSET_MODE = "earliest" 
TABLE_NAME = "kafka_data_stream" 
KAFKA_CONNECTOR_VERSION = "4.0.0" 
KAFKA_CONNECTOR_PACKAGE = f"org.apache.spark:spark-sql-kafka-0-10_2.13:{KAFKA_CONNECTOR_VERSION}"
COMMON_KAFKA_PACKAGE = "org.apache.kafka:kafka-clients:3.7.0"

## Initialisation de Spark Session

Création de la session Spark avec les connecteurs Kafka nécessaires.

In [ ]:
spark = SparkSession.builder \
    .appName("KafkaConsumerStream") \
    .config("spark.jars.packages", f"{KAFKA_CONNECTOR_PACKAGE},{COMMON_KAFKA_PACKAGE}") \
    .getOrCreate()

## Arrêt des streams existants

Vérification et arrêt des streams actifs portant le même nom pour éviter les conflits.

In [ ]:
for s in spark.streams.active:
    if s.name == TABLE_NAME and s.isActive:
        print(f"Arrêt du stream existant: {TABLE_NAME}")
        s.stop()
        time.sleep(1)  # Pause pour terminer proprement

## Configuration des logs

Réduction du niveau de logs pour plus de clarté.

In [ ]:
spark.sparkContext.setLogLevel("WARN")

## Connexion et lecture depuis Kafka

Configuration du stream Kafka et conversion des données en colonnes exploitables.

In [ ]:
print(f"Démarrage de la lecture depuis le topic '{TOPIC}' avec l'offset: {STARTING_OFFSET_MODE}")

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", BROKER_ADDRESS) \
    .option("startingOffsets", STARTING_OFFSET_MODE) \
    .option("subscribe", TOPIC) \
    .load() \
    .selectExpr(
        "CAST(key AS STRING) AS key", 
        "CAST(value AS STRING) AS value",
        "topic",
        "partition",
        "offset",
        "timestamp"
    )

## Écriture du stream en mémoire

Démarrage du stream et stockage des données dans une table temporaire en mémoire.

In [ ]:
query = df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(TABLE_NAME) \
    .start()

print(f"✅ Le stream est démarré et actif. Nom: {query.name}")

## Consultation des données (optionnel)

Pour visualiser les données en temps réel :

In [ ]:
spark.sql(f"SELECT * FROM {TABLE_NAME}").show()

## Statistiques du stream (optionnel)

In [ ]:
# Vérifier le statut
print(f"Stream actif: {query.isActive}")

# Voir les métriques
query.lastProgress

## Arrêt du stream

In [ ]:
# Décommenter pour arrêter le stream

# query.stop()
print('--THE END--')